In [ ]:
!pip install cerberus
!pip install validators

In [6]:
from cerberus import Validator
from validators import url

class Livro:
    __n_livros_cadastrados = 0
    __schema = {
        'titulo': {'type': 'string', 'minlength': 2, 'required': True},
        'autor': {'type': 'string', 'required': True},
        'ano_publicacao': {'type': 'integer', 'min': 0, 'max': 2025, 'required': True}
    }

    def __init__(self, titulo, autor, ano_publicacao):
        dados = {
            'titulo': titulo,
            'autor': autor,
            'ano_publicacao': ano_publicacao
        }
        
        v = Validator(Livro.__schema)
        if not v.validate(dados):
            raise ValueError(f"Dados inválidos: {v.errors}")

        self.titulo = titulo
        self.autor = autor
        self.ano_publicacao = ano_publicacao
        self.disp = True

        Livro.__n_livros_cadastrados += 1
        print(f"Livro {self.titulo} cadastrado!")

    # -------------- PROPERTIES ---------------

    @property
    def titulo(self):
        return self._titulo
    
    @titulo.setter
    def titulo(self, valor):
        schema = {'titulo': Livro.__schema["titulo"]}
        
        v = Validator(schema)
        if not v.validate({'titulo': valor}):
            raise ValueError(f"Título inválido: {v.errors}")
        
        self._titulo = valor

    @property
    def ano_publicacao(self):
        return self._ano_publicacao
    
    @ano_publicacao.setter
    def ano_publicacao(self, valor):
        schema = {'ano_publicacao': Livro.__schema["ano_publicacao"]}
        
        v = Validator(schema)
        if not v.validate({'ano_publicacao': valor}):
            raise ValueError(f"Ano inválido: {v.errors}")
        
        self._ano_publicacao = valor
    
    @property
    def autor(self):
        return self._autor
    
    @autor.setter
    def autor(self, valor):
        schema = {'autor': Livro.__schema["autor"]}
        
        v = Validator(schema)
        if not v.validate({'autor': valor}):
            raise ValueError(f"Autor inválido: {v.errors}")
        
        self._autor = valor

    #----------------------------------------------

    def emprestar(self) -> bool:
        # muda o estado de disponivel para False
        if self.disp: 
            self.disp = False

            print(f"Livro emprestado!")
            return True
        else: 
            print(f"Emprestar indisponível!")
            return False

    def devolver(self) -> bool:
        # muda o estado de disponivel para True
        if not self.disp: 
            self.disp = True

            print(f"Livro devolvido!")
            return True
        else: 
            print(f"Devolver indisponível!")
            return False

    def __str__(self):
        return f"{self.titulo}, {self.ano_publicacao} - {self.autor}"

    @classmethod
    def from_dict(cls, dados):
        # cria um uma instância a partir de um dict
        v = Validator(Livro.__schema)
        if not v.validate(dados):
            raise ValueError(f"Dados inválidos: {v.errors}")
        
        print(f"Livro {dados["titulo"]} criado")
        return cls(
            dados["titulo"],
            dados["autor"],
            dados["ano_publicacao"]
        )

    @classmethod
    def contar_livros(cls) -> int:
        # retorna o numero de livros cadastrados 
        return cls.__n_livros_cadastrados

class EBook(Livro):
    def __init__(self, titulo, autor, ano_publicacao, link_arquivo):
        if not url(link_arquivo):
            raise ValueError("Dados inválidos: link incorreto")
       
        super().__init__(titulo, autor, ano_publicacao)
        self.link_arquivo = link_arquivo

        print(f"Ebook {self.titulo} adicionado!")

    def emprestar(self):
        print("Link do Ebook encaminhado para o usuário!")
        return self.link_arquivo

    def devolver(self):
        raise AttributeError("Este método não é válido para Ebooks")

### ----------- TESTES PARA A CLASSE LIVRO -----------

In [8]:


print("\n=== TESTE 1: Criar Livro válido ===")
try:
    livro1 = Livro("O Hobbit", "J.R.R. Tolkien", 1937)
    print("✅ Sucesso:", livro1)
except Exception as e:
    print("❌ Falhou:", e)


print("\n=== TESTE 2: Criar Livro inválido (ano futuro) ===")
try:
    livroErro = Livro("Livro Futuro", "Autor", 3000)
    print("❌ ERRO! Isso não deveria ser válido")
except Exception as e:
    print("✅ Pegou erro corretamente:", e)


print("\n=== TESTE 3: Emprestar e devolver livro ===")
try:
    livro1.emprestar()
    print("Depois do empréstimo disp =", livro1.disp)
    livro1.devolver()
    print("Depois da devolução disp =", livro1.disp)
except Exception as e:
    print("❌ Falhou:", e)


print("\n=== TESTE 4: Criar via from_dict ===")
try:
    dados = {"titulo": "1984", "autor": "George Orwell", "ano_publicacao": 1949}
    livro2 = Livro.from_dict(dados)
    print("✅ Criado via dict:", livro2)
except Exception as e:
    print("❌ Falhou:", e)


print("\n=== TESTE 5: Contador de livros ===")
print("Livros cadastrados:", Livro.contar_livros())

# ----------- TESTES PARA A CLASSE EBOOK -----------

print("\n=== TESTE 6: Criar Ebook válido ===")
try:
    ebook1 = EBook("Python para Todos", "John Zelle", 2016, "https://pense-python.org")
    print("✅ Ebook criado:", ebook1)
except Exception as e:
    print("❌ Falhou:", e)


print("\n=== TESTE 7: Criar Ebook com URL inválida ===")
try:
    ebookErro = EBook("Falho", "Autor", 2000, "link_invalido")
    print("❌ ERRO! Isso não deveria ser válido")
except Exception as e:
    print("✅ Pegou erro corretamente:", e)


print("\n=== TESTE 8: Emprestar Ebook ===")
try:
    link = ebook1.emprestar()
    print("✅ Link enviado:", link)
except Exception as e:
    print("❌ Falhou:", e)


print("\n=== TESTE 9: Tentar devolver Ebook ===")
try:
    ebook1.devolver()
    print("❌ ERRO! Não deveria permitir devolver ebook")
except AttributeError as e:
    print("✅ Pegou erro corretamente:", e)



=== TESTE 1: Criar Livro válido ===
Livro O Hobbit cadastrado!
✅ Sucesso: O Hobbit, 1937 - J.R.R. Tolkien

=== TESTE 2: Criar Livro inválido (ano futuro) ===
✅ Pegou erro corretamente: Dados inválidos: {'ano_publicacao': ['max value is 2025']}

=== TESTE 3: Emprestar e devolver livro ===
Livro emprestado!
Depois do empréstimo disp = False
Livro devolvido!
Depois da devolução disp = True

=== TESTE 4: Criar via from_dict ===
Livro 1984 criado
Livro 1984 cadastrado!
✅ Criado via dict: 1984, 1949 - George Orwell

=== TESTE 5: Contador de livros ===
Livros cadastrados: 5

=== TESTE 6: Criar Ebook válido ===
Livro Python para Todos cadastrado!
Ebook Python para Todos adicionado!
✅ Ebook criado: Python para Todos, 2016 - John Zelle

=== TESTE 7: Criar Ebook com URL inválida ===
✅ Pegou erro corretamente: Dados inválidos: link incorreto

=== TESTE 8: Emprestar Ebook ===
Link do Ebook encaminhado para o usuário!
✅ Link enviado: https://pense-python.org

=== TESTE 9: Tentar devolver Ebook ===
